<a href="https://colab.research.google.com/github/varshachawan/PySparkApplication/blob/master/WindowingFunctions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Window functions

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://mirror.olnevhost.net/pub/apache/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

In [2]:
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"]="/content/spark-2.4.5-bin-hadoop2.7"
!echo $JAVA_HOME
import findspark
findspark.init()

/usr/lib/jvm/java-8-openjdk-amd64


In [0]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Window functions") \
    .getOrCreate()

In [0]:
products = spark.read\
                .format("csv")\
                .option("header", "true")\
                .load('./products.csv')

In [5]:
products.show()

+----------+--------+-----+
|   product|category|price|
+----------+--------+-----+
|Samsung TX|  Tablet|  999|
|Samsung JX|  Mobile|  799|
|Redmi Note|  Mobile|  399|
|        Mi|  Mobile|  299|
|      iPad|  Tablet|  789|
|    iPhone|  Mobile|  999|
|  Micromax|  Mobile|  249|
|    Lenovo|  Tablet|  499|
|   OnePlus|  Mobile|  356|
|        Xu|  Tablet|  267|
+----------+--------+-----+



#### Window rank function

In [0]:
import sys
from pyspark.sql.window import Window
import pyspark.sql.functions as func

In [0]:
windowSpec1 = Window \
    .partitionBy(products['category']) \
    .orderBy(products['price'].desc())

In [0]:
price_rank = (func.rank().over(windowSpec1))

In [15]:
product_rank = products.select(
        products['product'],
        products['category'],
        products['price']
).withColumn('rank', func.rank().over(windowSpec1))

product_rank.show()

+----------+--------+-----+----+
|   product|category|price|rank|
+----------+--------+-----+----+
|    iPhone|  Mobile|  999|   1|
|Samsung JX|  Mobile|  799|   2|
|   OnePlus|  Mobile|  356|   3|
|        Mi|  Mobile|  299|   4|
|Redmi Note|  Mobile|  249|   5|
|  Micromax|  Mobile|  249|   5|
|Samsung TX|  Tablet|  999|   1|
|      iPad|  Tablet|  499|   2|
|    Lenovo|  Tablet|  499|   2|
|        Xu|  Tablet|  267|   4|
+----------+--------+-----+----+



In [22]:
product_rank = products.select(
        products['product'],
        products['category'],
        products['price']
).withColumn('rank', func.dense_rank().over(windowSpec1))

product_rank.show()

+----------+--------+-----+----+
|   product|category|price|rank|
+----------+--------+-----+----+
|    iPhone|  Mobile|  999|   1|
|Samsung JX|  Mobile|  799|   2|
|   OnePlus|  Mobile|  356|   3|
|        Mi|  Mobile|  299|   4|
|Redmi Note|  Mobile|  249|   5|
|  Micromax|  Mobile|  249|   5|
|Samsung TX|  Tablet|  999|   1|
|      iPad|  Tablet|  499|   2|
|    Lenovo|  Tablet|  499|   2|
|        Xu|  Tablet|  267|   3|
+----------+--------+-----+----+



In [17]:
windowSpec0 = Window \
    .partitionBy(products['product']) \
    .orderBy(products['price'].desc())
product_rank = products.select(
        products['product'],
        products['category'],
        products['price'],
).withColumn('rank', func.rank().over(windowSpec0))

product_rank.show()

+----------+--------+-----+----+
|   product|category|price|rank|
+----------+--------+-----+----+
|        Xu|  Tablet|  267|   1|
|      iPad|  Tablet|  499|   1|
|Samsung JX|  Mobile|  799|   1|
|    iPhone|  Mobile|  999|   1|
|    Lenovo|  Tablet|  499|   1|
|  Micromax|  Mobile|  249|   1|
|   OnePlus|  Mobile|  356|   1|
|Redmi Note|  Mobile|  249|   1|
|Samsung TX|  Tablet|  999|   1|
|        Mi|  Mobile|  299|   1|
+----------+--------+-----+----+



#### Window max function between rows

In [0]:
windowSpec2 = Window \
    .partitionBy(products['category']) \
    .orderBy(products['price'].desc()) \
    .rowsBetween(-1, 0)

In [0]:
price_max = (func.max(products['price']).over(windowSpec2))

In [21]:
products.select(
    products['product'],
    products['category'],
    products['price'],
    price_max.alias("price_max")).show()

+----------+--------+-----+---------+
|   product|category|price|price_max|
+----------+--------+-----+---------+
|    iPhone|  Mobile|  999|      999|
|Samsung JX|  Mobile|  799|      999|
|   OnePlus|  Mobile|  356|      799|
|        Mi|  Mobile|  299|      356|
|Redmi Note|  Mobile|  249|      299|
|  Micromax|  Mobile|  249|      249|
|Samsung TX|  Tablet|  999|      999|
|      iPad|  Tablet|  499|      999|
|    Lenovo|  Tablet|  499|      499|
|        Xu|  Tablet|  267|      499|
+----------+--------+-----+---------+



#### Window price difference function between ranges

In [0]:
windowSpec3 = Window \
    .partitionBy(products['category']) \
    .orderBy(products['price'].desc()) \
    .rangeBetween(-sys.maxsize, sys.maxsize)

In [0]:
price_difference = \
  (func.max(products['price']).over(windowSpec3) - products['price'])

In [0]:
products.select(
    products['product'],
    products['category'],
    products['price'],
    price_difference.alias("price_difference")).show()

+----------+--------+-----+----------------+
|   product|category|price|price_difference|
+----------+--------+-----+----------------+
|    iPhone|  Mobile|  999|             0.0|
|Samsung JX|  Mobile|  799|           200.0|
|Redmi Note|  Mobile|  399|           600.0|
|   OnePlus|  Mobile|  356|           643.0|
|        Mi|  Mobile|  299|           700.0|
|  Micromax|  Mobile|  249|           750.0|
|Samsung TX|  Tablet|  999|             0.0|
|      iPad|  Tablet|  789|           210.0|
|    Lenovo|  Tablet|  499|           500.0|
|        Xu|  Tablet|  267|           732.0|
+----------+--------+-----+----------------+



In [0]:
windowSpec4 = Window \
    .partitionBy(products['category']) \
    .orderBy(products['price'].asc()) \
    .rangeBetween(0, sys.maxsize)

In [0]:
sys.maxsize

9223372036854775807

In [0]:
price_max = (func.max(products['price']).over(windowSpec4))

In [0]:
products.select(
    products['product'],
    products['category'],
    products['price'],
    price_max.alias("price_max")).show()

+----------+--------+-----+---------+
|   product|category|price|price_max|
+----------+--------+-----+---------+
|  Micromax|  Mobile|  249|      999|
|        Mi|  Mobile|  299|      999|
|   OnePlus|  Mobile|  356|      999|
|Redmi Note|  Mobile|  399|      999|
|Samsung JX|  Mobile|  799|      999|
|    iPhone|  Mobile|  999|      999|
|        Xu|  Tablet|  267|      999|
|    Lenovo|  Tablet|  499|      999|
|      iPad|  Tablet|  789|      999|
|Samsung TX|  Tablet|  999|      999|
+----------+--------+-----+---------+

